# Tema 12: Algoritmo de Shor

En este notebook vamos a implementar las funciones necesarias para realizar el algoritmo de Shor, uno de los algoritmos cuánticos más importantes y revolucionarios en el campo de la criptografía. 

El algoritmo de Shor permite factorizar números enteros grandes en sus factores primos de manera eficiente, algo que es computacionalmente intratable para ordenadores clásicos cuando los números son suficientemente grandes. Esta capacidad tiene implicaciones profundas para la seguridad de sistemas criptográficos como RSA, que dependen de la dificultad de factorizar números grandes.

Nuestro enfoque en este notebook estará principalmente en la **parte clásica** del algoritmo, que incluye:

- **Preparación del circuito cuántico**: Implementaremos las funciones necesarias para configurar el circuito cuántico que se utilizará en la parte cuántica del algoritmo. Esto incluye la selección de parámetros apropiados.

- **Procesamiento de resultados**: Desarrollaremos las funciones que toman la salida del circuito cuántico (típicamente en forma de mediciones de fase) y la procesan para extraer el período de la función modular. A partir de este período, aplicaremos técnicas clásicas de teoría de números para obtener los factores primos del número objetivo.

La parte cuántica, aunque crucial, será tratada en el siguiente tema. Nos centraremos en entender cómo preparar las entradas para esta parte cuántica y cómo interpretar sus salidas de manera efectiva.

## Librerías

In [ ]:
import numpy as np
import math
from fractions import Fraction

---
# Algoritmo general

El algoritmo de Shor es un algoritmo cuántico que permite factorizar un número compuesto $N = p \cdot q$ en sus factores primos $p$ y $q$ en tiempo polinómico, algo que los ordenadores clásicos no pueden hacer eficientemente para números grandes.

El algoritmo procede de la siguiente manera:

**Parte clásica:**

1. Elegir un número aleatorio $a$ entre $2$ y $N-1$.

2. Calcular $\gcd(a, N)$ (máximo común divisor).
   - Si $\gcd(a, N) \neq 1$, este número es un factor no trivial de $N$ y hemos terminado.

3. Si $\gcd(a, N) = 1$, usamos la parte cuántica para encontrar $r$, el período de la función:
   $$f(x) = a^x \bmod N$$

4. Este $r$ es el entero positivo más pequeño para el cual $f(x+r) = f(x)$.

5. Si $r$ es impar, volvemos al paso 1 (necesitamos que sea par para poder calcular $r/2$).

6. Si $a^{r/2} \equiv -1 \pmod{N}$, volvemos al paso 1.
   - En caso contrario, tanto $\gcd(a^{r/2}+1, N)$ como $\gcd(a^{r/2}-1, N)$ son factores no triviales de $N$, los primos que buscamos.

**Parte Cuántica:**

Hay dos enfoques posibles que estudiaremos más adelante, pero conceptualmente se basa en estudiar los autovalores del operador unitario $U_a$ que transforma $|x\rangle$ en $|(a \cdot x) \bmod N\rangle$. 

Este operador introduce una fase en dichos estados, la cual podemos medir mediante técnicas de estimación de fase cuántica (Quantum Phase Estimation). Esto nos permite obtener la periodicidad de la función $f(y) = a^y \bmod N$.

La idea fundamental es trabajar con una superposición de estados que entrelazaremos con el autoestado del operador. Como vamos a estudiar $f(y) = a^y \bmod N$, utilizaremos el estado de entrada $|1\rangle$, ya que es el autoestado natural de este operador.

---
# Funciones de la parte clásica


## Crea binario

Función que crea el binario de un número con el número necesario de bits

In [2]:
def numbin(a, n):
    '''Funcion que convierte el numero decimal a en uno binario con n bits.

    Parametros:
    a : entero que queremos pasar a binario.
    n : entero del numero de bits que queremos para el binario.
    '''
    a = bin(a)[2:]         #Pasamos el numero a binario
    return '0'*(n-len(a)) + a#Añadimos los ceros a la izquierda necesarios para que tenga el numero de bits.

## Inversión modular

Función que implementa a módulo inverso de N.

In [3]:
def m_inv(a, N):
    '''Funcion que implementa a modulo inverso de N.'''
    return pow(a, -1, N)

## Creador de N

Esta función genera el número **N** a partir de dos números primos de entrada **p** y **q**, donde N = p × q.

Además, la función busca un valor **a** válido para ejecutar el algoritmo de Shor. Este valor debe cumplir que mcd(a, N) = 1, es decir, que **a** y **N** sean coprimos.

**Parámetros importantes:**
- Podemos indicar qué valores de **a** ya hemos utilizado en llamadas anteriores mediante el parámetro `used_a`, para evitar repeticiones.
- Si mcd(a, N) ≠ 1, entonces hemos encontrado directamente un factor de N (lo cual es la solución que buscamos).
- Existe un límite de iteraciones (`itmax`) para evitar bucles infinitos en la búsqueda de un **a** válido.

In [4]:
def primador(p, q, itmax = 100, used_a = []):
    '''Funcion que implementa el crear un numero N a partir de 2 primos
    y nos da un numero a valido para el algoritmo.
    itmax es el numero de iteraciones permitidas para buscar este a.'''
    N   = p * q
    ver = True; itera = 0      #Comprueba que el maximo comun divisor de a y N sea 1.
    while ver and itera<itmax:
        a  = np.random.randint(2, N)#Entero aleatorio entre 2 y N.
        cd = np.gcd(a, N)
        if cd == 1 and (a not in used_a): ver = False
        itera += 1#Contador de iteraciones.
    if ver == True: print('No se ha encontrado un a valido.')
    return a, N

Hacemos una prueba

In [5]:
#Numeros primos para construir N.
p = 3
q = 7

#-------------------------------------
#Ejecutamos la funcion o los introducimos a mano.
a, N = primador(p, q)
print('a = ', a, ', N = ', N)
#Determinamos el numero de qbits n para el algoritmo (y añadimos uno extra)
n = 1 + int(np.floor(np.log2(N))); n +=1

a =  19 , N =  21


## Fracción irreducible

Esta función obtiene la **fracción irreducible** de un número racional.

La entrada será una fracción de la forma `num/den`, donde:
- `num` es el numerador
- `den` es el denominador

La función simplificará la fracción dividiéndola por su máximo común divisor, devolviendo así la fracción en su forma más reducida.

In [6]:
def reductor(num, den):
    '''Esta función implementa la obtención de
    la fraccion irreducible de un numero.
    El número de entrada va a ser una fraccion num/den.
    Ambos parametros son enteros.
    Se basa en dividir por el máximo común divisor.'''
    mcd = np.gcd(num, den)
    return num // mcd, den // mcd
    # Otra posible implementacion
    # for i in range(num, 0, -1):
    #     if num % i == 0 and den % i == 0:
    #         num /= i; den /= i
    # return int(num), int(den)

## Procesador de la salida

Esta función procesa los resultados obtenidos del algoritmo cuántico de Shor para extraer los factores primos de $N$.

**Proceso de extracción de factores:**

1. **Filtrado de resultados**: Obtenemos la lista de valores medidos $y$ que tienen una frecuencia superior a 0 en las mediciones. Estos valores se ordenan de mayor a menor frecuencia para priorizar los resultados más probables.

2. **Cálculo del periodo $r$**: Para cada valor $y$ obtenido, calculamos la fracción irreducible de $\frac{y}{2^t}$, siendo $t$ la precisión en bits de la estimación de fase (usualmente $t=2n$). El denominador de esta fracción irreducible nos proporciona un candidato para el periodo $r$ de la función $f(x) = a^x \bmod N$.

3. **Extracción de factores**: Una vez obtenido $r$, verificamos que sea par. Si lo es, calculamos los posibles factores usando:
   - $p = \gcd(a^{r/2} - 1, N)$
   - $q = \gcd(a^{r/2} + 1, N)$

4. **Validación**: Comprobamos que los valores obtenidos sean factores no triviales de $N$ (es decir, diferentes de 1 y de $N$).

In [ ]:
def procesador(counts, a, N, max_mult=None, reverse_bits=False):
    # Ordenar por frecuencia
    sorted_items = sorted(counts.items(), key=lambda x: x[1], reverse=True)

    # t: número de bits medidos (limpiando espacios). Usa el máximo por robustez.
    t = max(len(k.replace(" ", "")) for k in counts.keys())
    # Calcula 2^t usando desplazamiento de bits (equivalente a 2**t pero más eficiente)
    # El operador << desplaza 1 hacia la izquierda t posiciones, multiplicando por 2^t
    two_power_t = 1 << t

    # Cuántos múltiplos probar (por si el denominador recuperado es divisor del orden)
    if max_mult is None:
        max_mult = min(N, 2 * t + 10)

    for key, freq in sorted_items:
        bits = key.replace(" ", "")
        if len(bits) != t:
            # Si hay longitudes distintas, ajusta a t por la izquierda (típico) o ignora
            bits = bits.zfill(t)

        if reverse_bits:
            bits = bits[::-1]

        x = int(bits, 2)
        if x == 0:
            continue

        # Aproximación tipo fracciones continuas: x/2^t ~ s/r0 con r0 <= N
        frac = Fraction(x, two_power_t).limit_denominator(N)
        r0 = frac.denominator
        if r0 <= 0:
            continue

        # Probar múltiplos de r0 y validar el periodo
        for m in range(1, max_mult + 1):
            r = m * r0
            if r > N:
                break

            if pow(a, r, N) != 1:
                continue
            if r % 2 != 0:
                continue

            ar2 = pow(a, r // 2, N)
            if ar2 == N - 1:
                continue

            p = math.gcd(ar2 - 1, N)
            q = math.gcd(ar2 + 1, N)

            if 1 < p < N:
                return p, N // p
            if 1 < q < N:
                return q, N // q

    return 1, 1